In [9]:
# regex search pattern, matching columns will be sorted to the top
pattern = ""

# copy paste the columns or tables to drill into below
drilldown = """

"""

In [10]:
## view schema

from dotenv import dotenv_values
import polars as pl
import os.path
table = dotenv_values("../.env")["DESCRIBE"]

# load schema
drilldown = list(filter(None, map(lambda col: col.split(':')[0].strip(" └─🗝️🔑✅\t"), drilldown.splitlines())))
if table:
    schema = pl.read_parquet(f"./{table}.parquet")
    
    # query schema
    columns = schema.select(
            "column_name",
            pl.col("column_name").str.contains(pattern).alias("match"),
            pl.col("column_name").is_in(drilldown).alias("drilldown"),
            "column_desc",
            "type_name",
            "oa_length",
            "oa_precision",
            "oa_radix",
            "oa_scale",
            "pktable_name",
            "pkcolumn_name",
            "fk_name",
            pl.col("pk_name").is_not_null().alias('pk'),
            pl.when(pl.col("pk_name").is_not_null())\
                .then(pl.lit('🔑'))\
                .when(pl.col("fk_name").is_not_null())\
                .then(pl.lit('🗝️'))\
                .otherwise(pl.lit(""))\
                .alias("key")
        )\
        .unique()\
        .sort( #is unique optional now?
            ["drilldown", "pk", "match", "column_name"],
            descending=[True, True, True, False]
        )\
        .rows()

    # print schema
    print(schema.select(pl.col("table_qualifier") + "\n└─ " + pl.col("table_name"))[0][0].item() + f" ({len(columns)} Columns)")
    for col in columns:
        print(f"  └─{col[-1][-2:]} {col[0]}{': ' + '.'.join(col[9:11]) if col[11] is not None else ''}")
        if col[2]: # drilldown
            for field in zip(col[3:], ["", "", "length: ", "precision: ", "radix: ", "scale: "]):
                if field[0] is not None:
                    print(f"{'    ' if len(field[1]) > 0 else ''}    └─ {field[1]}{field[0]}")

else:
    schema = pl.read_parquet(f"./object_explorer.parquet")

    objects = schema.select(
            "table_qualifier",
            "table_name",
            "remarks",
            pl.col("table_name").str.contains(pattern).alias("match"),
            pl.col("table_name").is_in(drilldown).alias("drilldown")
        )\
        .sort(
            ["drilldown", "match", "table_qualifier", "table_name"],
            descending=[True, True, True, False]
        )\
        .rows()
    
    qualifier = None
    for obj in objects:
        if obj[0] != qualifier:
            if qualifier != None:
                print()
            print(obj[0])
            qualifier = obj[0]
        
        downloaded = '\t✅' if os.path.exists(f'./{obj[1]}.parquet') else ''
        print(f"└─ {obj[1]}{downloaded}")

        if obj[-1]:
            print(f"    └─ {obj[2]}")          


Continental Battery Company
└─ transaction (555 Columns)
  └─🔑 id
  └─ abbrevtype
  └─ accountbasednumber
  └─ actionitem
  └─ actualproductionenddate
  └─ actualproductionstartdate
  └─ actualshipdate
  └─ altsalestotal
  └─🗝️ approvalstatus: ApprovalStatus.id
  └─ balsegstatus
  └─🗝️ billingaddress: transactionBillingAddress.nkey
  └─ billingstatus
  └─ buyingreason
  └─ buyingtimeframe
  └─ closedate
  └─ committed
  └─ completeddate
  └─🗝️ createdby: entity.id
  └─🗝️ createdby: employee.id
  └─🗝️ createdby: Partner.id
  └─🗝️ createdby: Vendor.id
  └─🗝️ createdby: Customer.id
  └─ createddate
  └─🗝️ currency: currency.id
  └─ custbody2
  └─🗝️ custbody_10184_customer_entity_bank: CUSTOMRECORD_2663_ENTITY_BANK_DETAILS.id
  └─🗝️ custbody_11187_pref_entity_bank: CUSTOMRECORD_2663_ENTITY_BANK_DETAILS.id
  └─ custbody_11724_bank_fee
  └─ custbody_11724_pay_bank_fees
  └─🗝️ custbody_15529_emp_entity_bank: CUSTOMRECORD_2663_ENTITY_BANK_DETAILS.id
  └─🗝️ custbody_15529_vendor_entity_bank: CU